# Sentiment and Thematic Analyses of British Politician Speeches

#### Kanav Bhagat 730013149

## importaing important libraries that will be used for the experimentations

In [15]:
import pandas as pd
import nltk

from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams

import spacy
from gensim import corpora
from gensim.models import LdaModel
import gensim
from nltk import pos_tag


import matplotlib.pyplot as plt
import seaborn as sns

## Downloading important modules from nltk library

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')


## Loading the Dataset into notebook

In [ ]:
#Load the Dataset into datafram
Speech_data = pd.read_csv('SpeechData_with_Party.csv')
print(Speech_data.head())

'''
# Counting the number of speeches by Conservative, Labour, and Other parties
speech_count_by_party = Speech_data['Party'].value_counts()
print("Number of speeches by each party:\n", speech_count_by_party)
'''

### Speeches Dataset Overview

Below is a preview of the dataset containing speeches from British politicians, categorized by their name, title, URL, text, and party affiliation.

| Politician Name | Speech Title                                             | Speech URL                                                        | Party        |
|-----------------|----------------------------------------------------------|-------------------------------------------------------------------|--------------|
| Scott Benton    | 2024 Resignation Statement                               | [Link](https://www.ukpol.co.uk/scott-benton-2024-resignation...)  | Other        |
| Rishi Sunak     | 2024 Speech on Extremism                                 | [Link](https://www.ukpol.co.uk/rishi-sunak-2024-speech-on-ext...)  | Conservative |
| Michael Gove    | 2024 Speech at the Convention of the North               | [Link](https://www.ukpol.co.uk/michael-gove-2024-speech-at-th...)  | Conservative |
| Oliver Dowden   | 2024 Speech on AI for Public Good                        | [Link](https://www.ukpol.co.uk/oliver-dowden-2024-speech-on-a...)  | Conservative |
| Stuart Andrew   | 2024 Speech at the Beacon Philanthropy and Impact Forum  | [Link](https://www.ukpol.co.uk/stuart-andrew-2024-speech-at-t...)  | Conservative |

This table provides a quick look at the diversity of topics and party representation in our dataset. Each speech's URL is linked for easy access to the full text.


## Performing Sentiment Analysis

In [18]:
#Calling the sentiment Analyser 
sia = SentimentIntensityAnalyzer()

def analyze_sentiment_vader(text):
    sentiment_score = sia.polarity_scores(text)
    return sentiment_score

In [19]:
#Applying the Sentiment Analysis over the speeches. 
#No need to apply any pre-processing since Vader is efficient to work over human speeches. 
sentiment_df = Speech_data['Speech Text'].apply(analyze_sentiment_vader)
Speech_data[['Neg', 'Neu', 'Pos', 'Compound']] = pd.json_normalize(sentiment_df) #Adding new columns to the dataframe for generating results 

In [ ]:
#Finding Average compound of all the speeches
average_compound = Speech_data['Compound'].mean()
print(f"Average Compound Sentiment Score: {average_compound:.3f}")

### Average Compound Sentiment Score

The overall average compound sentiment score across all analyzed speeches is **0.808**. This score indicates a generally positive sentiment within the political speeches in our dataset.


### Analysing the data using the graphs


In [ ]:
#Plotting Sentiment distribution across all the speeches.
plt.figure(figsize=(4, 3))
sns.histplot(Speech_data['Compound'], bins=30, kde=True, color='skyblue')
plt.title('Distribution of Compound Sentiment Scores Across All Speeches')
plt.xlabel('Compound Score')
plt.ylabel('Number of Speeches')
plt.show()

#### Graph representing Distribution of Compound Sentiment Scores Across All Speeches 
![Graph Results](Markdown%20Images/graph-1.png "Graph showing Distribution of Compound Sentiment Scores Across All Speeches")


In [ ]:
# Calculating average sentiment scores for each political party
avg_sentiment_by_party = Speech_data.groupby('Party')[['Compound']].mean().reset_index()

print(avg_sentiment_by_party)

plt.figure(figsize=(9, 3))
plt.subplot(1, 2, 2)
sns.barplot(x='Party', y='Compound', data=avg_sentiment_by_party, palette='Purples')
plt.title('Average Compound Sentiment')
plt.xlabel('Party')
plt.ylabel('Average Score')

### Results for sentiments based upon different parties

Below is the summary of average compound sentiment scores by party:

| Party         | Compound |
|---------------|----------|
| Conservative  | 0.884162 |
| Labour        | 0.467271 |
| Other         | 0.290737 |

### Visual Representation

For a more detailed analysis, refer to the graph below:

![Sentiment Analysis Results](Markdown%20Images/graph-2.png "Sentiment Analysis Graph")


In [ ]:
# Identifying speeches with the highest and lowest compound scores
highest_sentiment_speech = Speech_data.loc[Speech_data['Compound'].idxmax()]
lowest_sentiment_speech = Speech_data.loc[Speech_data['Compound'].idxmin()]

print(f"Highest Sentiment Speech: '{highest_sentiment_speech['Speech Title']}' by {highest_sentiment_speech['Politician Name']} from {highest_sentiment_speech['Party']} (Compound: {highest_sentiment_speech['Compound']})")
print(f"Lowest Sentiment Speech: '{lowest_sentiment_speech['Speech Title']}' by {lowest_sentiment_speech['Politician Name']} from {highest_sentiment_speech['Party']} (Compound: {lowest_sentiment_speech['Compound']})")


### Speech Sentiment Extremes

#### Highest Sentiment Speech
- **Title:** 2024 Budget Speech
- **Politician:** Jeremy Hunt
- **Party:** Conservative
- **Sentiment Score (Compound):** 1.0

#### Lowest Sentiment Speech
- **Title:** 2024 Speech on Freedom and Democracy in Iran
- **Politician:** Bob Blackman
- **Party:** Conservative
- **Sentiment Score (Compound):** -0.9998


## Performing Themetic Analysis

In [24]:
nlp = spacy.load("en_core_web_sm")

#Funtion preprocess_text pre-processes the text data by various different ways.
"""
- It lowercase the text, 
- Remove the first sentence since first sentence of each speech tells the name of politician, title of speech and date it was delivered.
- Tokenize the text
- Remove non-alphanumeric characters and stopwords. Added a list of common stopwords in english to get us the most refined themes. 
- Lemmatize the text
- Generate bigrams and multi-word texts 

"""
def preprocess_text(text):
    text = text.lower()
    doc = nlp(text)
    entities = [entity.text for entity in doc.ents]
    # Combine entity tokens back into the text
    text_with_entities = ' '.join(entities) + ' ' + text
    
    # Remove the first sentence (usually metadata in political speeches)
    sentences = nltk.sent_tokenize(text_with_entities)
    if len(sentences) > 1:
        text = ' '.join(sentences[1:])
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove non-alphanumeric characters and stopwords
    common_stop_words = {"also","new","year","0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz",}
    stop_words = set(stopwords.words('english')).union(common_stop_words)
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Generate and add bigrams to capture multi-word themes
    bigrams = ['_'.join(gram) for gram in ngrams(tokens, 2)]
    tokens.extend(bigrams)
    
    return ' '.join(tokens)

### Preprocessing the speeches based upon the political party

In [25]:
#Pre-process the speeches based upon the political party
Speech_data['Titile and Speech'] = Speech_data['Speech Title'] + ". " + Speech_data['Speech Text']

pre_processed_conservative= Speech_data[Speech_data['Party'] == 'Conservative']['Titile and Speech'].apply(preprocess_text)
pre_processed_labour= Speech_data[Speech_data['Party'] == 'Labour']['Titile and Speech'].apply(preprocess_text)
pre_processed_other= Speech_data[Speech_data['Party'] == 'Other']['Titile and Speech'].apply(preprocess_text)


### Function to process texts before we can apply LDA. 


In [26]:
#Funtion to process texts before we can apply LDA. 
#This function builds the dictionary and a corpus from the speech data to be used for LDA
def process_speeches(preprocessed_speeches):
    # Tokenize the preprocessed speeches
    # Initialize an empty list to hold the filtered tokens
    texts = []

    for speech in preprocessed_speeches:
        # Tokenize the preprocessed speech
        tokens = word_tokenize(speech)
        
        # Perform POS tagging on the tokens
        tagged_tokens = pos_tag(tokens)
        
        # Filter tokens based on POS tags (keeping only nouns and verbs as an example)
        allowed_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
        important_words = [word for word, tag in tagged_tokens if tag in allowed_pos_tags]
        
        texts.append(important_words)

    # Create a dictionary representation of the documents, and filter extremes
    dictionary = corpora.Dictionary(texts)
    dictionary.filter_extremes(no_below=2, no_above=0.9)

    # Convert document into the bag-of-words (BoW) format
    corpus = [dictionary.doc2bow(text) for text in texts]

    if len(corpus) == 0 or len(dictionary) == 0:
        print("Warning: Corpus or dictionary is empty. Consider adjusting filtering criteria.")
    
    
    return corpus, dictionary

### LDA function to perform themetic Analysis


In [27]:
#Function to create LDA model with corpus and dictionary. We will generate 5 topics for each party corpus.
def create_lda_model(corpus, dictionary, num_topics=5, passes=15):
    model = LdaModel(
        corpus=corpus,
        id2word=dictionary,
        chunksize=100,
        alpha='auto',
        eta='auto',
        iterations=400,
        num_topics=num_topics,
        passes=passes
    )

    # Get topics from the model
    topics = model.print_topics(num_words=4)
    
    return topics

### Getting the themes for different parties using LDA

In [28]:
#Applying the LDA to our speeches filtered upon parties. 

corpus_conservative, dictionary_conservative, = process_speeches(pre_processed_conservative)
Keywords_conservative = create_lda_model(corpus_conservative, dictionary_conservative, num_topics=5)

corpus_labour, dictionary_labour, = process_speeches(pre_processed_labour)
Keywords_labour = create_lda_model(corpus_labour, dictionary_labour, num_topics=5)

corpus_other, dictionary_other, = process_speeches(pre_processed_other)
Keywords_other = create_lda_model(corpus_other, dictionary_other, num_topics=5)

In [ ]:
#Printing the results
print("Conservative")
for i in Keywords_conservative:
    print(i)

print("Labour")
for i in Keywords_labour:
    print(i)

print("Other")
for i in Keywords_other:
    print(i)


## Thematic Analysis Results

### Conservative Themes
1. `0.012*"people" + 0.010*"investment" + 0.010*"business" + 0.009*"work"`
2. `0.010*"ireland" + 0.007*"northern_ireland" + 0.007*"government" + 0.007*"defence"`
3. `0.014*"gambling" + 0.008*"government" + 0.008*"ireland" + 0.008*"consultation"`
4. `0.016*"woman" + 0.013*"government" + 0.012*"people" + 0.009*"service"`
5. `0.013*"food" + 0.009*"country" + 0.008*"child" + 0.008*"today"`

### Labour Themes
1. `0.085*"government" + 0.055*"authority" + 0.047*"funding" + 0.036*"care"`
2. `0.060*"people" + 0.058*"politics" + 0.046*"country" + 0.042*"britain"`
3. `0.077*"ireland" + 0.060*"northern_ireland" + 0.042*"minister" + 0.030*"market"`
4. `0.039*"member" + 0.029*"government" + 0.029*"hon_member" + 0.026*"house"`
5. `0.005*"house" + 0.005*"member" + 0.005*"government" + 0.005*"hope"`

### Other Party Themes
1. `0.045*"continue" + 0.045*"return" + 0.045*"comment" + 0.024*"life"`
2. `0.051*"community" + 0.051*"mp" + 0.041*"government" + 0.041*"country"`
3. `0.064*"house" + 0.045*"party" + 0.039*"opposition" + 0.039*"leader"`
4. `0.064*"office" + 0.049*"meet" + 0.049*"minister" + 0.034*"court"`
5. `0.075*"iran" + 0.034*"freedom" + 0.021*"people" + 0.021*"member"`
